In [ ]:
!pip install -q duckduckgo-search
!pip install -q serpapi
!pip install -q groq

import os
import requests
import serpapi
from duckduckgo_search import DDGS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 3.2 MB/s eta 0:00:00


In [ ]:
os.environ["GROQ_API_KEY"] = "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"
os.environ["SERPAPI_API_KEY"] = "1de1687c638882a90b7947583842eae9ed49874efccf36ab9c96b8c2ec330ebe"

In [ ]:
def buscar_no_ddg(consulta, max_resultados=3):
    """Realiza uma busca no DuckDuckGo e retorna os resultados formatados."""
    try:
        with DDGS() as ddgs:
            resultados = ddgs.text(consulta, max_results=max_resultados)
            if not resultados:
                return "Nenhum resultado encontrado no DuckDuckGo."
            return "\n".join([f"- {r.get('title', '')}: {r.get('body', '')}" for r in resultados])
    except Exception as e:
        return f"Erro ao buscar no DuckDuckGo: {e}"

def buscar_com_serpapi(consulta, num_resultados=3):
    """Realiza uma busca no Google usando a API da SerpApi e retorna os resultados formatados."""
    try:
        parametros = {
            "q": consulta,
            "api_key": os.environ["SERPAPI_API_KEY"],
            "engine": "google",
            "num": num_resultados
        }
        client = serpapi.Client()
        resultados = client.search(parametros)

        if "organic_results" not in resultados:
            return "Nenhum resultado orgânico encontrado no Google."

        snippets = []
        for item in resultados.get("organic_results", []):
            title = item.get("title", "")
            snippet = item.get("snippet", "")
            snippets.append(f"- {title}: {snippet}")

        return "\n".join(snippets)
    except Exception as e:
        return f"Erro ao buscar com SerpApi: {e}"

In [ ]:
def gerar_resposta_com_groq(comportamento, tema, dados_ddg, dados_google, modelo="llama3-70b-8192", temperatura=0.7):
    """Gera uma resposta utilizando a API da Groq com base no comportamento, tema e dados de busca."""
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {os.environ['GROQ_API_KEY']}",
        "Content-Type": "application/json"
    }

    system_prompt = comportamento
    user_prompt = f"""
    Com base nas informações coletadas abaixo, responda sobre o tema '{tema}'.

    Informações do DuckDuckGo:
    {dados_ddg}

    Informações do Google:
    {dados_google}
    """

    payload = {
        "model": modelo,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "temperature": temperatura
    }

    try:
        resposta = requests.post(url, headers=headers, json=payload)
        resposta.raise_for_status()
        return resposta.json()['choices'][0]['message']['content']
    except requests.exceptions.RequestException as e:
        return f"Erro na chamada da API da Groq: {e}"
    except (KeyError, IndexError) as e:
        return f"Erro ao processar a resposta da Groq: {e}, Resposta: {resposta.text}"

In [ ]:
def processar_testes(testes):
    """Processa a lista de testes, busca informações e gera as respostas."""
    lista_respostas = []
    for teste in testes:
        if not teste:
            continue

        id_teste = teste.get("id")
        comportamento = teste.get("comportamento")
        tema = teste.get("tema")

        if not all([id_teste, comportamento, tema]):
            print(f"Aviso: Teste com id '{id_teste}' está incompleto e será ignorado.")
            continue

        print(f"Processando id: {id_teste} | Tema: {tema}")
        print("Buscando no DuckDuckGo...")
        info_ddg = buscar_no_ddg(tema)

        print("Buscando no Google (SerpApi)...")
        info_google = buscar_com_serpapi(tema)
        print("Gerando resposta com a Groq...")
        resposta_final = gerar_resposta_com_groq(comportamento, tema, info_ddg, info_google)

        lista_respostas.append({
            'id': id_teste,
            'resposta': resposta_final
        })
        print("-" * 30)

    return lista_respostas

In [ ]:
testes = [
    {
        "id": "1",
        "comportamento": "Aja como um especialista em nutrição que responde de maneira simples e acessível",
        "tema": "Como alimentos podem interferir na glicose"
    },
    {
        "id": "2",
        "comportamento": "Aja como um especialista em nutrição que responde de forma acadêmica e formal",
        "tema": "Como alimentos podem interferir na glicose"
    },
    {
        "id": "3",
        "comportamento": "Aja como um especialista em marketing e responda como um influenciador de rede social",
        "tema": "Mix de marketing"
    },
    {
        "id": "4",
        "comportamento": "Aja como um especialista em marketing e responda de maneira acadêmica e formal",
        "tema": "Mix de marketing"
    }
]
resultado_final = processar_testes(testes)

Processando id: 1 | Tema: Como alimentos podem interferir na glicose
Buscando no DuckDuckGo...
Buscando no Google (SerpApi)...
Gerando resposta com a Groq...
------------------------------
Processando id: 2 | Tema: Como alimentos podem interferir na glicose
Buscando no DuckDuckGo...
Buscando no Google (SerpApi)...
Gerando resposta com a Groq...
------------------------------
Processando id: 3 | Tema: Mix de marketing
Buscando no DuckDuckGo...
Buscando no Google (SerpApi)...
Gerando resposta com a Groq...
------------------------------
Processando id: 4 | Tema: Mix de marketing
Buscando no DuckDuckGo...
Buscando no Google (SerpApi)...
Gerando resposta com a Groq...
------------------------------


In [ ]:
print("\n--- RESULTADO FINAL ---")
for resultado in resultado_final:
    print(f"ID: {resultado['id']}")
    print("RESPOSTA:")
    print(resultado['resposta'])
    print("=" * 60)


--- RESULTADO FINAL ---
ID: 1
RESPOSTA:
Olá!

Vou explicar de forma simples como os alimentos podem interferir na glicose.

**Alimentos que devem ser evitados:**

1. **Alimentos ricos em açúcar**: Doces, bolos, tortas, bebidas doces, como cerveja e vinho, podem aumentar rapidamente a glicose no sangue.
2. **Alimentos ricos em gordura**: Gorduras saturadas e trans podem aumentar o risco de doenças cardíacas e renais.
3. **Alimentos processados**: Conservas, alimentos enlatados e processados têm alto teor de sódio, o que pode contribuir para a hipertensão.
4. **Bebidas alcoólicas**: Álcool pode interferir na regulação da glicose e aumentar o risco de diabetes.

**Alimentos que podem ajudar:**

1. **Alimentos ricos em fibras**: Vegetais, frutas, legumes e cereais integrais ajudam a controlar a glicose no sangue.
2. **Proteínas e gorduras boas**: Consumir proteínas e gorduras saudáveis, como aves, peixes, nozes e sementes, antes de consumir carboidratos pode ajudar a controlar a glicose.
